## 第8章 文埋め込み

### 文埋め込みモデルの実装

#### 教師なしSimCSEの実装

In [1]:
from transformers.trainer_utils import set_seed

# 乱数シードの設定
set_seed(42)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### データセットの読み込みと前処理

In [2]:
from datasets import load_dataset

unsup_train_dataset = load_dataset(
    "llm-book/jawiki-sentences", split="train"
)

Using custom data configuration default
Reusing dataset jawiki-sentences (/root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc)


In [3]:
# 訓練セットの形式と事例数を確認
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 24387500
})


In [4]:
# 訓練セットの中身を確認する
for i, text in enumerate(unsup_train_dataset[:50]["text"]):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

In [5]:
# 訓練セットから空白行を事例を削除
unsup_train_dataset = unsup_train_dataset.filter(
    lambda example: example["text"].strip() != ""
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-ecc223a34f1ffe88.arrow


In [6]:
# 訓練セットをシャッフルし、最初の100万事例を取り出す
unsup_train_dataset = unsup_train_dataset.shuffle().select(
    range(1000000)
)
# パフォーマンスの低下を防ぐために、シャッフルされた状態の訓練セットを
# ディスクに書き込む
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-e9e6caaca5ef995f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-9484a91dba79bc01.arrow


In [7]:
# 前処理後の訓練セットの形式と事例数を確認
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


In [8]:
# 前処理後の訓練セットの内容を確認
for i, text in enumerate(unsup_train_dataset[:10]["text"]):
    print(i, text)

0 2005年の時点で、10,000人ものウズベキスタン人が韓国での労働に従事しており、その大部分が高麗人である。
1 小学5年生(11歳)の時から芸能活動を開始。
2 i ħ d d t | ψ ( t ) ⟩ = L ^ | ψ ( t ) ⟩ {\displaystyle i\hbar {\frac {d}{dt}}|\psi (t)\rangle ={\hat {L}}|\psi (t)\rangle }
3 安土宗論(あづちしゅうろん)は、1579年(天正7年)、安土城下の浄厳院で行われた浄土宗と法華宗の宗論。
4 1927年 オーストラリア選手権(1927ねんオーストラリアせんしゅけん、1927 Australian Championships)に関する記事。
5 さらにマップ上で最大8つまでしか建築できず(司令官アビリティの”解体”か設置したプレイヤー自らが出向いて解体する必要がある)
6 特に誉淳が1827年から作成した『古瓦譜』は畿内で600点以上の拓本を蒐集し、瓦当文様に着目したうえで編年を試みている。
7 マルクス主義者を広言し、メキシコ共産党の敵であり味方であった。
8 ICHILLIN'(アイチリン、朝: 아이칠린)は、韓国の7人組女性アイドルグループ。
9 マークVIは1983年にモデルサイクルを終了し、1984年のマークVII(英語版)はフルサイズセグメントから撤退し、マークシリーズは異なるセグメントに移行した。


In [9]:
# Hugging Face Hubのllm-book/JGLUEのリポジトリから
# JSTSデータセットの訓練セットと検証セットを読み込み、
# それぞれをSimCSEの検証セットとテストセットとして使用する
valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="train"
)
test_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="validation"
)

Reusing dataset jglue (/root/.cache/huggingface/datasets/llm-book___jglue/JSTS/1.1.0/b394a8dbefe82fb1dc2724c1eb79bb1ea3062df2037f91a69a27c089f3ff685f)
Reusing dataset jglue (/root/.cache/huggingface/datasets/llm-book___jglue/JSTS/1.1.0/b394a8dbefe82fb1dc2724c1eb79bb1ea3062df2037f91a69a27c089f3ff685f)


#### トークナイザとcollate関数の準備

In [10]:
from transformers import AutoTokenizer

base_model_name = "tohoku-nlp/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [58]:
import torch
from torch import Tensor
from transformers import BatchEncoding

def unsup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """教師なしSimCSEの訓練セットのミニバッチを作成"""
    # ミニバッチに含まれる文にトークナイザを適用する
    tokenized_texts = tokenizer(
        [example["text"] for example in examples],
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文）に対してi列目の事例（文）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    return {
        "tokenized_texts_1": tokenized_texts,
        "tokenized_texts_2": tokenized_texts,
        "labels": labels,
    }

In [59]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """SimCSEの検証・テストセットのミニバッチを作成"""
    # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれに
    # トークナイザを適用する
    tokenized_texts_1 = tokenizer(
        [example["sentence1"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized_texts_2 = tokenizer(
        [example["sentence2"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文1）に対して
    # i列目の事例（文2）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    # データセットに付与された類似度スコアのTensorを作成する
    label_scores = torch.tensor(
        [example["label"] for example in examples]
    )

    return {
        "tokenized_texts_1": tokenized_texts_1,
        "tokenized_texts_2": tokenized_texts_2,
        "labels": labels,
        "label_scores": label_scores,
    }

#### モデル準備

In [60]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSEModel(nn.Module):
    """SimCSEのモデル"""

    def __init__(
        self,
        base_model_name: str,
        mlp_only_train: bool = False,
        temperature: float = 0.05,
    ):
        """モデルの初期化"""
        super().__init__()

        # モデル名からエンコーダを初期化する
        self.encoder = AutoModel.from_pretrained(base_model_name)
        # パラメータをメモリ上に隣接した形で配置
        # これを実行しない場合、モデルの保存でエラーになることがある
        # for param in model.parameters():
        #     param.data = param.data.contiguous()
        # MLP層の次元数
        self.hidden_size = self.encoder.config.hidden_size
        # MLP層の線形層
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        # MLP層の活性化関数
        self.activation = nn.Tanh()

        # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
        self.mlp_only_train = mlp_only_train
        # 交差エントロピー損失の計算時に使用する温度
        self.temperature = temperature

    def encode_texts(self, tokenized_texts: BatchEncoding) -> Tensor:
        """エンコーダを用いて文をベクトルに変換"""
        # トークナイズされた文をエンコーダに入力する
        encoded_texts = self.encoder(**tokenized_texts)
        # モデルの最終層の出力（last_hidden_state）の
        # [CLS]トークン（0番目の位置のトークン）のベクトルを取り出す
        encoded_texts = encoded_texts.last_hidden_state[:, 0]

        # self.mlp_only_trainのフラグがTrueに設定されていて
        # かつ訓練時でない場合、MLP層の変換を適用せずにベクトルを返す
        if self.mlp_only_train and not self.training:
            return encoded_texts

        # MLP層によるベクトルの変換を行う
        encoded_texts = self.dense(encoded_texts)
        encoded_texts = self.activation(encoded_texts)

        return encoded_texts

    def forward(
        self,
        tokenized_texts_1: BatchEncoding,
        tokenized_texts_2: BatchEncoding,
        labels: Tensor,
        label_scores: Tensor | None = None,
    ) -> ModelOutput:
        """モデルの前向き計算を定義"""
        # 文ペアをベクトルに変換する
        encoded_texts_1 = self.encode_texts(tokenized_texts_1)
        encoded_texts_2 = self.encode_texts(tokenized_texts_2)

        # 文ペアの類似度行列を作成する
        sim_matrix = F.cosine_similarity(
            encoded_texts_1.unsqueeze(1),
            encoded_texts_2.unsqueeze(0),
            dim=2,
        )

        # 交差エントロピー損失を求める
        loss = F.cross_entropy(sim_matrix / self.temperature, labels)

        # 性能評価に使用するため、正例ペアに対するスコアを類似度行列から取り出す
        positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
        positive_scores = torch.masked_select(
            sim_matrix, positive_mask
        )

        return ModelOutput(loss=loss, scores=positive_scores)

# 教師なしSimCSEのモデルを初期化する
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

#### Trainerの準備

In [69]:
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
    """
    モデルが予測したスコアと評価用データのスコアの
    スピアマンの順位相関係数を計算
    """
    scores = p.predictions
    labels, label_scores = p.label_ids

    spearman = spearmanr(scores, label_scores).correlation

    return {"spearman": spearman}

In [74]:
from transformers import TrainingArguments

# 教師なしSimCSEの訓練のハイパーパラメータを設定する
unsup_training_args = TrainingArguments(
    output_dir="../model/outputs_unsup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=256,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=256,  # 評価時のバッチサイズ
    learning_rate=3e-5,  # 学習率
    num_train_epochs=1,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
    report_to="none",  # 外部ツールへのログを無効化
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [75]:
# !pip install transformers[torch]

In [76]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import Trainer

class SimCSETrainer(Trainer):
    """SimCSEの訓練に使用するTrainer"""

    def get_eval_dataloader(
        self, eval_dataset: Dataset | None = None
    ) -> DataLoader:
        """
        検証・テストセットのDataLoaderでeval_collate_fnを使うように
        Trainerのget_eval_dataloaderをオーバーライド
        """
        if eval_dataset is None:
            eval_dataset = self.eval_dataset

        return DataLoader(
            eval_dataset,
            batch_size=64,
            collate_fn=eval_collate_fn,
            pin_memory=True,
        )

# 教師なしSimCSEのTrainerを初期化する
unsup_trainer = SimCSETrainer(
    model=unsup_model,
    args=unsup_training_args,
    data_collator=unsup_train_collate_fn,
    train_dataset=unsup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

####  訓練の実行

In [77]:
# 教師なしSimCSEの訓練を行う
unsup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,0.000400,2.331667,0.752163
500,0.000500,2.402906,0.755473
750,0.000400,2.385067,0.753259
1000,0.000300,2.366536,0.757948
1250,0.000400,2.350700,0.759100
1500,0.000800,2.320323,0.760896
1750,0.000400,2.332596,0.759427
2000,0.000200,2.334367,0.757139
2250,0.000300,2.300360,0.758956
2500,0.000100,2.297671,0.758454


TrainOutput(global_step=3907, training_loss=0.00033567894401759603, metrics={'train_runtime': 1629.5441, 'train_samples_per_second': 613.669, 'train_steps_per_second': 2.398, 'total_flos': 0.0, 'train_loss': 0.00033567894401759603, 'epoch': 1.0})

#### 性能評価

In [78]:
# 検証セットで教師なしSimCSEのモデル評価を行う
unsup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.3203227519989014,
 'eval_spearman': 0.7608961210671525,
 'eval_runtime': 10.878,
 'eval_samples_per_second': 1144.601,
 'eval_steps_per_second': 4.504,
 'epoch': 1.0}

In [79]:
# テストセットで教師なしSimCSEのモデル評価を行う
unsup_trainer.evaluate(test_dataset)

{'eval_loss': 2.1774518489837646,
 'eval_spearman': 0.7878933714566898,
 'eval_runtime': 1.3524,
 'eval_samples_per_second': 1077.307,
 'eval_steps_per_second': 4.436,
 'epoch': 1.0}

In [80]:
#### トークナイザの保存とモデルの保存
encoder_path = "../model/outputs_unsup_simcse/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

('../model/outputs_unsup_simcse/encoder/tokenizer_config.json',
 '../model/outputs_unsup_simcse/encoder/special_tokens_map.json',
 '../model/outputs_unsup_simcse/encoder/vocab.txt',
 '../model/outputs_unsup_simcse/encoder/added_tokens.json')

#### 教師ありSimCSEの実装